# Home Credit Default Risk — Data Preparation (Colab / Python 3)
# This notebook builds a reproducible data preparation pipeline:
# - Fits parameters on training data only (no leakage)
# - Applies identical transformations to train and test
# - Aggregates bureau + previous_application to SK_ID_CURR
# - Joins aggregated features back to application tables

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

In [2]:
app_train = pd.read_csv("application_train.csv")
app_test  = pd.read_csv("application_test.csv")
bureau    = pd.read_csv("bureau.csv")
prev      = pd.read_csv("previous_application.csv")
inst      = pd.read_csv("installments_payments.csv")

print("Train:", app_train.shape)
print("Test:", app_test.shape)
print("Bureau:", bureau.shape)
print("Prev:", prev.shape)
print("Inst:", inst.shape)

print("\nSanity checks:")
print("TARGET in train:", "TARGET" in app_train.columns)
print("TARGET in test :", "TARGET" in app_test.columns)
print("SK_ID_CURR in all:", all("SK_ID_CURR" in df.columns for df in [app_train, app_test, bureau, prev, inst]))


Train: (307511, 122)
Test: (48744, 121)
Bureau: (1716428, 17)
Prev: (1670214, 37)
Inst: (13605401, 8)

Sanity checks:
TARGET in train: True
TARGET in test : False
SK_ID_CURR in all: True


In [3]:
def safe_div(num, den):
    # Avoid divide-by-zero
    den = den.replace({0: np.nan}) if isinstance(den, pd.Series) else (np.nan if den == 0 else den)
    return num / den

def add_missing_flags(df, cols):
    df = df.copy()
    for c in cols:
        if c in df.columns:
            df[f"{c}_missing"] = df[c].isna().astype("int8")
    return df

In [4]:
def fit_prep_params(app_train: pd.DataFrame):
    params = {}

    # EXT_SOURCE medians learned from train only
    ext_cols = [c for c in ["EXT_SOURCE_1","EXT_SOURCE_2","EXT_SOURCE_3"] if c in app_train.columns]
    params["ext_medians"] = {c: float(app_train[c].median(skipna=True)) for c in ext_cols}

    # Train-only bin edges for age and employment
    age_years = (-app_train["DAYS_BIRTH"] / 365.25)
    emp_days  = app_train["DAYS_EMPLOYED"].replace(365243, np.nan)
    emp_years = (-emp_days / 365.25)

    def quantile_bins(x, qs=(0, .2, .4, .6, .8, 1)):
        edges = np.unique(np.nanquantile(x, qs))
        if len(edges) < 4:
            edges = np.unique(np.linspace(np.nanmin(x), np.nanmax(x), 6))
        return edges.tolist()

    params["age_bins"] = quantile_bins(age_years.values)
    params["emp_bins"] = quantile_bins(emp_years.values)

    # Categorical levels (train only). Unseen levels -> "Unknown"
    cat_cols = app_train.select_dtypes(include=["object"]).columns.tolist()
    levels_map = {}
    for c in cat_cols:
        s = app_train[c].fillna("Missing").astype(str)
        lv = sorted(s.unique().tolist())
        if "Unknown" not in lv:
            lv.append("Unknown")
        levels_map[c] = lv
    params["cat_levels"] = levels_map

    return params

params = fit_prep_params(app_train)
params.keys()

dict_keys(['ext_medians', 'age_bins', 'emp_bins', 'cat_levels'])

In [5]:
def apply_cat_levels(df: pd.DataFrame, params):
    df = df.copy()
    levels_map = params.get("cat_levels", {})
    for c, lv in levels_map.items():
        if c not in df.columns:
            continue
        s = df[c].fillna("Missing").astype(str)
        s = s.where(s.isin(lv), other="Unknown")
        df[c] = pd.Categorical(s, categories=lv)
    return df

In [6]:
def transform_application(app: pd.DataFrame, params: dict):
    df = app.copy()

    # Fix known anomaly
    if "DAYS_EMPLOYED" in df.columns:
        df["DAYS_EMPLOYED"] = df["DAYS_EMPLOYED"].replace(365243, np.nan)
        df["employed_missing"] = df["DAYS_EMPLOYED"].isna().astype("int8")

    # Demographics in years (interpretable)
    df["age_years"] = (-df["DAYS_BIRTH"] / 365.25)
    if "DAYS_EMPLOYED" in df.columns:
        df["employ_years"] = (-df["DAYS_EMPLOYED"] / 365.25)

    # Missing flags
    df = add_missing_flags(df, ["EXT_SOURCE_1","EXT_SOURCE_2","EXT_SOURCE_3","AMT_ANNUITY","AMT_GOODS_PRICE","OWN_CAR_AGE"])

    # Impute EXT_SOURCE using TRAIN medians
    for c, med in params.get("ext_medians", {}).items():
        if c in df.columns:
            df[c] = df[c].fillna(med)

    # Financial ratios
    df["credit_to_income"]  = safe_div(df["AMT_CREDIT"], df["AMT_INCOME_TOTAL"])
    df["annuity_to_income"] = safe_div(df["AMT_ANNUITY"], df["AMT_INCOME_TOTAL"])
    df["credit_to_annuity"] = safe_div(df["AMT_CREDIT"], df["AMT_ANNUITY"])
    df["goods_to_credit"]   = safe_div(df["AMT_GOODS_PRICE"], df["AMT_CREDIT"])

    # Bins learned from train only
    df["age_bin"] = pd.cut(df["age_years"], bins=params["age_bins"], include_lowest=True)
    if "employ_years" in df.columns:
        df["employ_bin"] = pd.cut(df["employ_years"], bins=params["emp_bins"], include_lowest=True)

    # Simple interaction
    if all(c in df.columns for c in ["EXT_SOURCE_1","EXT_SOURCE_2"]):
        df["ext12_interaction"] = df["EXT_SOURCE_1"] * df["EXT_SOURCE_2"]

    # Apply train category levels
    df = apply_cat_levels(df, params)

    return df

train_t = transform_application(app_train, params)
test_t  = transform_application(app_test, params)

print("Transformed train:", train_t.shape)
print("Transformed test :", test_t.shape)

Transformed train: (307511, 138)
Transformed test : (48744, 137)


In [7]:
def aggregate_bureau(bureau_df: pd.DataFrame):
    b = bureau_df.copy()
    b["is_active"] = (b["CREDIT_ACTIVE"] == "Active").astype("int8")
    b["is_closed"] = (b["CREDIT_ACTIVE"] == "Closed").astype("int8")

    agg = b.groupby("SK_ID_CURR", as_index=False).agg(
        bureau_n_credits=("SK_ID_BUREAU","count") if "SK_ID_BUREAU" in b.columns else ("CREDIT_ACTIVE","size"),
        bureau_n_active=("is_active","sum"),
        bureau_n_closed=("is_closed","sum"),
        bureau_overdue_sum=("AMT_CREDIT_SUM_OVERDUE","sum"),
        bureau_overdue_max=("AMT_CREDIT_SUM_OVERDUE","max"),
        bureau_debt_sum=("AMT_CREDIT_SUM_DEBT","sum"),
        bureau_debt_max=("AMT_CREDIT_SUM_DEBT","max"),
        bureau_credit_sum=("AMT_CREDIT_SUM","sum")
    )

    agg["bureau_debt_to_credit"] = safe_div(agg["bureau_debt_sum"], agg["bureau_credit_sum"])
    agg["bureau_overdue_to_credit"] = safe_div(agg["bureau_overdue_sum"], agg["bureau_credit_sum"])
    return agg

bureau_agg = aggregate_bureau(bureau)
print("bureau_agg:", bureau_agg.shape)
bureau_agg.head()

bureau_agg: (305811, 11)


,SK_ID_CURR,bureau_n_credits,bureau_n_active,bureau_n_closed,bureau_overdue_sum,bureau_overdue_max,bureau_debt_sum,bureau_debt_max,bureau_credit_sum,bureau_debt_to_credit,bureau_overdue_to_credit
0,100001,7,3,4,0.0,0.0,596686.5,373239.0,1453365.000,0.410555,0.0
1,100002,8,2,6,0.0,0.0,245781.0,245781.0,865055.565,0.284122,0.0
2,100003,4,1,3,0.0,0.0,0.0,0.0,1017400.500,0.000000,0.0
3,100004,2,0,2,0.0,0.0,0.0,0.0,189037.800,0.000000,0.0
4,100005,3,2,1,0.0,0.0,568408.5,543087.0,657126.000,0.864992,0.0


In [8]:
def aggregate_previous_application(prev_df: pd.DataFrame):
    p = prev_df.copy()
    p["is_approved"] = (p["NAME_CONTRACT_STATUS"] == "Approved").astype("int8")
    p["is_refused"]  = (p["NAME_CONTRACT_STATUS"] == "Refused").astype("int8")

    agg = p.groupby("SK_ID_CURR", as_index=False).agg(
        prev_n_apps=("SK_ID_PREV","count") if "SK_ID_PREV" in p.columns else ("NAME_CONTRACT_STATUS","size"),
        prev_n_approved=("is_approved","sum"),
        prev_n_refused=("is_refused","sum"),
        prev_amt_app_mean=("AMT_APPLICATION","mean"),
        prev_amt_credit_mean=("AMT_CREDIT","mean")
    )

    agg["prev_approval_rate"] = safe_div(agg["prev_n_approved"], agg["prev_n_apps"])
    agg["prev_refusal_rate"]  = safe_div(agg["prev_n_refused"],  agg["prev_n_apps"])
    return agg

prev_agg = aggregate_previous_application(prev)
print("prev_agg:", prev_agg.shape)
prev_agg.head()

prev_agg: (338857, 8)


,SK_ID_CURR,prev_n_apps,prev_n_approved,prev_n_refused,prev_amt_app_mean,prev_amt_credit_mean,prev_approval_rate,prev_refusal_rate
0,100001,1,1,0,24835.50,23787.00,1.0,0.0
1,100002,1,1,0,179055.00,179055.00,1.0,0.0
2,100003,3,3,0,435436.50,484191.00,1.0,0.0
3,100004,1,1,0,24282.00,20106.00,1.0,0.0
4,100005,2,1,0,22308.75,20076.75,0.5,0.0


In [9]:
# --- Aggregate installments_payments.csv to applicant-level (SK_ID_CURR) ---
# Since you can upload installments_payments.csv into Colab, we can read it directly and aggregate it.

def aggregate_installments(inst_df: pd.DataFrame) -> pd.DataFrame:
    i = inst_df.copy()

    # Row-level features
    i["days_late"] = i["DAYS_ENTRY_PAYMENT"] - i["DAYS_INSTALMENT"]
    i["is_late"] = (i["days_late"] > 0).fillna(False).astype("int8")
    i["days_late_pos"] = i["days_late"].clip(lower=0)

    denom = i["AMT_INSTALMENT"].replace({0: np.nan})
    i["pay_ratio"] = i["AMT_PAYMENT"] / denom

    # Applicant-level aggregation
    agg = i.groupby("SK_ID_CURR", as_index=False).agg(
        inst_n_payments=("SK_ID_PREV", "count") if "SK_ID_PREV" in i.columns else ("AMT_PAYMENT", "size"),
        inst_late_pct=("is_late", "mean"),
        inst_days_late_mean=("days_late_pos", "mean"),
        inst_pay_ratio_mean=("pay_ratio", "mean"),
        inst_pay_ratio_min=("pay_ratio", "min"),
        inst_pay_ratio_max=("pay_ratio", "max"),
    )

    return agg

inst_agg = aggregate_installments(inst)
print("inst_agg:", inst_agg.shape)
inst_agg.head()


inst_agg: (339587, 7)


,SK_ID_CURR,inst_n_payments,inst_late_pct,inst_days_late_mean,inst_pay_ratio_mean,inst_pay_ratio_min,inst_pay_ratio_max
0,100001,7,0.142857,1.571429,1.0,1.0,1.0
1,100002,19,0.000000,0.000000,1.0,1.0,1.0
2,100003,25,0.000000,0.000000,1.0,1.0,1.0
3,100004,3,0.000000,0.000000,1.0,1.0,1.0
4,100005,9,0.111111,0.111111,1.0,1.0,1.0


In [10]:
# Join aggregations to Train/Test (bureau + previous_application + installments)

train_joined = (
    train_t
    .merge(bureau_agg, on="SK_ID_CURR", how="left")
    .merge(prev_agg,   on="SK_ID_CURR", how="left")
    .merge(inst_agg,   on="SK_ID_CURR", how="left")
)

test_joined = (
    test_t
    .merge(bureau_agg, on="SK_ID_CURR", how="left")
    .merge(prev_agg,   on="SK_ID_CURR", how="left")
    .merge(inst_agg,   on="SK_ID_CURR", how="left")
)

print("Train joined:", train_joined.shape)
print("Test joined :", test_joined.shape)


Train joined: (307511, 161)
Test joined : (48744, 160)


In [11]:
def align_train_test(train_df: pd.DataFrame, test_df: pd.DataFrame, target_col="TARGET"):
    train = train_df.copy()
    test  = test_df.copy()

    y = None
    if target_col in train.columns:
        y = train[target_col].copy()
        train = train.drop(columns=[target_col])

    if target_col in test.columns:
        test = test.drop(columns=[target_col])

    all_cols = sorted(set(train.columns).union(set(test.columns)))

    for c in all_cols:
        if c not in train.columns:
            train[c] = np.nan
        if c not in test.columns:
            test[c] = np.nan

    train = train[all_cols]
    test  = test[all_cols]

    if y is not None:
        train = pd.concat([y.rename(target_col), train], axis=1)

    return train, test

train_final, test_final = align_train_test(train_joined, test_joined)

print("Final train:", train_final.shape)
print("Final test :", test_final.shape)

train_cols = [c for c in train_final.columns if c != "TARGET"]
print("Columns match (train vs test):", sorted(train_cols) == sorted(test_final.columns))

Final train: (307511, 161)
Final test : (48744, 160)
Columns match (train vs test): True


In [13]:
# Convert Interval columns to strings because PyArrow/Parquet doesn't support them natively
for col in ["age_bin", "employ_bin"]:
    if col in train_final.columns:
        train_final[col] = train_final[col].astype(str)
    if col in test_final.columns:
        test_final[col] = test_final[col].astype(str)

train_final.to_parquet("/content/train_prepared.parquet", index=False)
test_final.to_parquet("/content/test_prepared.parquet", index=False)

print("Saved:")
!ls -lh /content/*prepared* /content/*.parquet

Saved:
-rw-r--r-- 1 root root 8.4M Mar  2 01:33 /content/test_prepared.parquet
-rw-r--r-- 1 root root 8.4M Mar  2 01:33 /content/test_prepared.parquet
-rw-r--r-- 1 root root  45M Mar  2 01:33 /content/train_prepared.parquet
-rw-r--r-- 1 root root  45M Mar  2 01:33 /content/train_prepared.parquet
